In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weghted, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weghted['item_id'])
    items_weights = np.array(items_weghted['weight'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights)
    
    return recs.tolist()

In [5]:
items_weghted = data_train[['item_id', 'sales_value']].groupby(['item_id'], as_index=False).sum()

items_weghted['sales_value'] = items_weghted['sales_value'].apply(lambda x: 0 if x <= 1 else np.log(x)) # отр.рез. зануляем
items_weghted['sales_value'] = items_weghted['sales_value']/ (items_weghted['sales_value']).sum()
items_weghted.rename(columns={'sales_value': 'weight'}, inplace=True)
items_weghted['weight'].sum()

0.9999999999999999

Проверяем, чтобы по итогам не было отрицательного веса (для этого выше занулили получающиеся отрицательные значения)

In [6]:
items_weghted.loc[items_weghted['weight'] < 0]

,item_id,weight


In [7]:
items_weghted.head(3)

,item_id,weight
0,25671,0.000013
1,26081,0.000000
2,26093,0.000002


In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [9]:
%%time

# your_code

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weghted, n=5))

result.head(2)

Wall time: 5.01 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]"


In [10]:
items_weghted['weight'].median()

1.044496070230781e-05

In [11]:
items_weghted.loc[items_weghted['item_id'].isin([
    999391, 1131625, 10344586, 6534178, 12754036, 908504, 6534178, 1069506, 6961519, 921514
])]['weight'] > items_weghted['weight'].median()

15510    True
16982    True
25553    True
33262    True
40052    True
55470    True
57258    True
70817    True
76292    True
Name: weight, dtype: bool

В рекомендациях первым двум пользователям товары с бОльшим весом чем медианное значение.

In [12]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [13]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [14]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

Wall time: 5.7 s


,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]","[1040199, 12605392, 856221, 2418699, 944454]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]","[970747, 1100856, 1339110, 9526269, 15717033]"


In [15]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [16]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [17]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [18]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

C:\Users\LMAXL\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [19]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()



In [20]:
user_item_matrix.shape

(2499, 5001)

In [21]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [22]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [23]:
# your_code 

### 1.1a  Home work: Random recommendation in TOP 5K items

In [24]:
%%time

items = data_train.item_id.unique()

result['random_recommendation of TOP 5000'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))

result.head(2)

Wall time: 1.31 s


,user_id,actual,weighted_random_recommendation,random_recommendation,random_recommendation of TOP 5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]","[1040199, 12605392, 856221, 2418699, 944454]","[5592610, 1000672, 12731436, 907892, 1139525]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]","[970747, 1100856, 1339110, 9526269, 15717033]","[9524291, 870327, 879876, 977658, 1067419]"


### Weighted Random recommendation in TOP 5K items

In [25]:
items_weghted_5000 = data_train[['item_id', 'sales_value']].groupby(['item_id'], as_index=False).sum()
items_weghted_5000 = items_weghted_5000.loc[items_weghted_5000['item_id'].isin(top_5000)]
items_weghted_5000['sales_value'] = items_weghted_5000['sales_value'].apply(lambda x: 0 if x <= 1 else np.log(x)) # отр.рез. зануляем
items_weghted_5000['sales_value'] = items_weghted_5000['sales_value']/ (items_weghted_5000['sales_value']).sum()
items_weghted_5000.rename(columns={'sales_value': 'weight'}, inplace=True)
items_weghted_5000['weight'].sum()

1.0

In [26]:
%%time

# your_code

result['weighted_random_recommendation of TOP 5K'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weghted_5000, n=5))

result.head(2)

Wall time: 529 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,random_recommendation of TOP 5000,weighted_random_recommendation of TOP 5K
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]","[1040199, 12605392, 856221, 2418699, 944454]","[5592610, 1000672, 12731436, 907892, 1139525]","[978354, 1058939, 5569379, 1013826, 1065017]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]","[970747, 1100856, 1339110, 9526269, 15717033]","[9524291, 870327, 879876, 977658, 1067419]","[1075470, 872417, 6463789, 5578547, 820486]"


### 1.2 Popularity-based recommendation

In [27]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [28]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 172 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,random_recommendation of TOP 5000,weighted_random_recommendation of TOP 5K,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]","[1040199, 12605392, 856221, 2418699, 944454]","[5592610, 1000672, 12731436, 907892, 1139525]","[978354, 1058939, 5569379, 1013826, 1065017]","[6666, 6534178, 6533889, 1029743, 6534166]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]","[970747, 1100856, 1339110, 9526269, 15717033]","[9524291, 870327, 879876, 977658, 1067419]","[1075470, 872417, 6463789, 5578547, 820486]","[6666, 6534178, 6533889, 1029743, 6534166]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Доп: отфильтровать мусорный товар во всех рекомендациях

## 2.1 Item-Item Recommender / ItemKNN

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Доп: отфильтровать мусорный товар во всех рекомендациях

In [29]:
%%time
for K in range (1,10,2):
    
    model = ItemItemRecommender(K, num_threads=4) # K - кол-во билжайших соседей


    model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
              show_progress=True)
    
    result['ItemItem, no 6666, K=' + str(K)] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 8.56 s


In [30]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.95 s


In [31]:
recs

[(0, 78679.0),
 (3408, 72173.0),
 (2149, 57995.0),
 (3947, 17272.0),
 (3587, 14417.0)]

In [32]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 1127831, 1098066]

In [33]:
%%time

result['itemitem all, K=5'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

Wall time: 120 ms


In [34]:
result.head(5)

,user_id,actual,weighted_random_recommendation,random_recommendation,random_recommendation of TOP 5000,weighted_random_recommendation of TOP 5K,popular_recommendation,"ItemItem, no 6666, K=1","ItemItem, no 6666, K=3","ItemItem, no 6666, K=5","ItemItem, no 6666, K=7","ItemItem, no 6666, K=9","itemitem all, K=5"
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]","[1040199, 12605392, 856221, 2418699, 944454]","[5592610, 1000672, 12731436, 907892, 1139525]","[978354, 1058939, 5569379, 1013826, 1065017]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[6666, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]","[970747, 1100856, 1339110, 9526269, 15717033]","[9524291, 870327, 879876, 977658, 1067419]","[1075470, 872417, 6463789, 5578547, 820486]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[6666, 1082185, 981760, 1098066, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[838662, 8067176, 854031, 1011771, 12672832]","[839853, 12605066, 5980738, 1185318, 1004510]","[835243, 1118046, 8014645, 1388206, 1062572]","[1043095, 901666, 1116635, 5591520, 5568228]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 995242, 1098066, 1127831]","[1082185, 981760, 995242, 840361, 1098066]","[6666, 1082185, 981760, 1127831, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1034478, 991886, 992668, 934172, 6704149]","[1149843, 2640727, 837694, 1810202, 41919]","[5571248, 9835408, 9575181, 878302, 1117336]","[12670635, 845537, 926903, 917839, 1112238]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 995242, 1029743, 826249, 1127831]","[1082185, 981760, 995242, 1029743, 826249]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1098066, 995242, 1127831]","[1082185, 981760, 995242, 1098066, 826249]","[6666, 1082185, 981760, 1127831, 995242]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[963117, 888764, 985055, 12325406, 932410]","[841016, 67136, 855441, 1204179, 13037614]","[8015407, 8119303, 5995312, 9526624, 7025490]","[997988, 972416, 1004585, 957772, 1020581]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 6534178, 995242, 1029743]","[1082185, 981760, 1127831, 1098066, 995242]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 995242, 826249]","[6666, 1082185, 981760, 1127831, 1098066]"


### 4.2 Косинусное сходство и CosineRecommender

In [35]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.85 s


In [36]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 6666, 981760, 1127831, 1098066]

In [37]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 124 ms


Cosine rec без 6666

In [38]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[6666]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.93 s


In [39]:
%%time

result['cosine no 6666'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

Wall time: 142 ms


In [40]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,random_recommendation of TOP 5000,weighted_random_recommendation of TOP 5K,popular_recommendation,"ItemItem, no 6666, K=1","ItemItem, no 6666, K=3","ItemItem, no 6666, K=5","ItemItem, no 6666, K=7","ItemItem, no 6666, K=9","itemitem all, K=5",cosine,cosine no 6666
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]","[1040199, 12605392, 856221, 2418699, 944454]","[5592610, 1000672, 12731436, 907892, 1139525]","[978354, 1058939, 5569379, 1013826, 1065017]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]","[970747, 1100856, 1339110, 9526269, 15717033]","[9524291, 870327, 879876, 977658, 1067419]","[1075470, 872417, 6463789, 5578547, 820486]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 1098066, 981760, 826249, 883404]"


### 4.3 TF-IDF взвешивание и TFIDFRecommender

In [41]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 2.03 s


In [42]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 981760, 1127831, 6666, 1098066]

In [43]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 153 ms


TF-IDF без 6666

In [44]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[6666]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.87 s


In [45]:
%%time

result['tfidf no 6666'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=False)])

Wall time: 144 ms


### 4.5 Измерим качество по precision@5

In [47]:
result.head(5)

,user_id,actual,weighted_random_recommendation,random_recommendation,random_recommendation of TOP 5000,weighted_random_recommendation of TOP 5K,popular_recommendation,"ItemItem, no 6666, K=1","ItemItem, no 6666, K=3","ItemItem, no 6666, K=5","ItemItem, no 6666, K=7","ItemItem, no 6666, K=9","itemitem all, K=5",cosine,cosine no 6666,tfidf,tfidf no 6666
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569201, 2287297, 973880, 13115515, 6773187]","[1040199, 12605392, 856221, 2418699, 944454]","[5592610, 1000672, 12731436, 907892, 1139525]","[978354, 1058939, 5569379, 1013826, 1065017]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042564, 915545, 1074754, 1499569, 15483133]","[970747, 1100856, 1339110, 9526269, 15717033]","[9524291, 870327, 879876, 977658, 1067419]","[1075470, 872417, 6463789, 5578547, 820486]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 981760, 1098066, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[838662, 8067176, 854031, 1011771, 12672832]","[839853, 12605066, 5980738, 1185318, 1004510]","[835243, 1118046, 8014645, 1388206, 1062572]","[1043095, 901666, 1116635, 5591520, 5568228]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 995242, 1098066, 1127831]","[1082185, 981760, 995242, 840361, 1098066]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 6666, 878996]","[1082185, 981760, 1127831, 878996, 961554]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1034478, 991886, 992668, 934172, 6704149]","[1149843, 2640727, 837694, 1810202, 41919]","[5571248, 9835408, 9575181, 878302, 1117336]","[12670635, 845537, 926903, 917839, 1112238]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 995242, 1029743, 826249, 1127831]","[1082185, 981760, 995242, 1029743, 826249]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1098066, 995242, 1127831]","[1082185, 981760, 995242, 1098066, 826249]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 6666, 1127831, 961554]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 6666, 961554]","[1082185, 981760, 1127831, 961554, 826249]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[963117, 888764, 985055, 12325406, 932410]","[841016, 67136, 855441, 1204179, 13037614]","[8015407, 8119303, 5995312, 9526624, 7025490]","[997988, 972416, 1004585, 957772, 1020581]","[6666, 6534178, 6533889, 1029743, 6534166]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 6534178, 995242, 1029743]","[1082185, 981760, 1127831, 1098066, 995242]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 995242, 826249]","[6666, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981

In [48]:
# Функции из 1-ого вебинара
import os, sys
    
from metrics import precision_at_k, recall_at_k

# Metrics

In [49]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0009:weighted_random_recommendation
0.0007:random_recommendation
0.0063:random_recommendation of TOP 5000
0.0068:weighted_random_recommendation of TOP 5K
0.0856:popular_recommendation


C:\Users\LMAXL\Documents\обучение\15 рек системы\курс 07.2021\webinar_2\metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


0.2199:ItemItem, no 6666, K=1
0.219:ItemItem, no 6666, K=3
0.151:ItemItem, no 6666, K=5
0.16:ItemItem, no 6666, K=7
0.1642:ItemItem, no 6666, K=9
0.1368:itemitem all, K=5
0.1329:cosine
0.1551:cosine no 6666
0.139:tfidf
0.1609:tfidf no 6666


#### Выводы:

Исключение непопулярных товаров увеличивает метрику, улучшает результат.

Для ItemItem лучшее K = 1, далее 3 и 9 , это же и топ3 модели по результатам.

